In [1]:
import numpy as np
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(42)
random.seed(42)

from DataExploration import DataExploration
from DecisionTree import DecisionTree
from NeuralNetwork import DNN
from Boosting import Boosting
from KNN import KNN
from NeuralNetworkKeras import NN_Keras
from SVM import SVM


C:\Users\matan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:

#need to shuffle to respect iid principal for when we split the data
de = DataExploration()

#twitter_df = de.prepare_twitter_df()
speed_dating_df = de.prepare_speed_dating_df()
encoded_speed_dating_df = de.get_hot_encoded_speed_dating(speed_dating_df)

(8378, 44)


C:\Users\matan\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


removing empty values shape:
(6794, 44)


In [5]:
#print(encoded_speed_dating_df.head())
#print(twitter_df.describe())
print(speed_dating_df.describe())

         decision  gender_encoded  race_encoded  race_o_encoded  field_encoded
count  6794.00000     6794.000000   6794.000000     6794.000000    6794.000000
mean      0.43141        0.508979      2.152782        2.154843     145.596556
std       0.49531        0.499956      1.357241        1.347755      68.424596
min       0.00000        0.000000      0.000000        0.000000       0.000000
25%       0.00000        0.000000      1.000000        1.000000      88.000000
50%       0.00000        1.000000      3.000000        3.000000     173.000000
75%       1.00000        1.000000      3.000000        3.000000     201.000000
max       1.00000        1.000000      4.000000        4.000000     247.000000


In [ ]:
X_train, X_test, y_train, y_test, X_train_nn, X_validation_nn, y_train_nn, y_validation_nn = de.get_train_test_validation(
    encoded_speed_dating_df, 'speed_dating')


In [9]:
print(np.any(np.isnan(X_train)))
print(np.all(np.isfinite(X_train)))
print(X_train.shape)

False
True
(4382, 299)


In [9]:
# # data processing and investigation
# TODO for speed dating
# data_twitter = twitter_df.drop('created_at', axis=1).drop('collected_at', axis=1).drop(
#                 'series_of_num_following', axis=1)
# data_twitter['label'] = np.where(data_twitter['is_spam'] == 1, 'spam', 'not_spam')
# data_twitter = data_twitter.drop('is_spam', axis=1)
# pos_twitter = data_twitter.loc[data_twitter['label'] == 'spam'].drop('label', axis=1)
# neg_twitter = data_twitter.loc[data_twitter['label'] == 'not_spam'].drop('label', axis=1)
# 
# create_scatterplot_matrix(data_twitter, "label", 'twitter_df')
# plot_violin_distributions(data_twitter, title='Violin Plots of Twitter Features', dataset_name='twitter_df', label_column='label')
# compare_counts_boxplots(positive_pd=pos_twitter, negative_pd=neg_twitter, title='Boxplots of Twitter Features', positive_label='spam', dataset_name='twitter_df')


In [10]:
#best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7} #twitter
best_params = {'dt__alpha': -0.1, 'dt__max_depth': 9, 'dt__min_samples_leaf': 2} #speed_dating  (CV score=0.724):
dt_clf = DecisionTree(model_name='dt_wip1', dataset_name='speed_dating', X_test=X_test, y_test=y_test, best_params=best_params)
dt_clf.train(X_train, y_train)
y_pred = dt_clf.predict(X_test)
dt_clf.evaluate_model(y_test=y_test, y_pred=y_pred)

#dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth


[[515 119]
 [147 315]]
             precision    recall  f1-score   support

        0.0       0.78      0.81      0.79       634
        1.0       0.73      0.68      0.70       462

avg / total       0.76      0.76      0.76      1096



In [11]:
dt_best_params = dt_clf.get_best_params()
print(dt_best_params)
#print(dt_clf.feature_importance_)

{'dt__alpha': -0.1, 'dt__max_depth': 9, 'dt__min_samples_leaf': 2}


In [12]:
# dnn = DNN(n_classes=2, model_name='dnn_wip_3', dataset_name='spam_twitter')
# dnn.train(
#     learning_rate=0.05,
#     steps=1000,
#     batch_size=30,
#     hidden_units=[100, 100],
#     training_examples=X_train_nn,
#     training_targets=y_train_nn,
#     validation_examples=X_validation_nn,
#     validation_targets=y_validation_nn)
# 
# y_pred = dnn.predict(X_test, y_test)

In [12]:
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 40} (CV score=0.772):
# todo this is lower than pre pruning, figure out why
boosting_clf = Boosting(model_name='boosting_wip0', dataset_name='speed_dating', X_test=X_test, y_test=y_test)
boosting_clf.train(X_train, y_train)
y_pred = boosting_clf.predict(X_test)
boosting_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Best parameter (CV score=0.772):
{'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 40}


[[509 125]
 [151 311]]
             precision    recall  f1-score   support

        0.0       0.77      0.80      0.79       634
        1.0       0.71      0.67      0.69       462

avg / total       0.75      0.75      0.75      1096



In [14]:
boosting_best_params = boosting_clf.get_best_params()
print(boosting_best_params)

{'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 40}


In [15]:
#best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'}
knn_clf = KNN(model_name='knn_wip0', dataset_name='speed_dating')
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)
#{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'} (CV score=0.728):

Best parameter (CV score=0.728):
{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'}


[[527 107]
 [171 291]]
             precision    recall  f1-score   support

        0.0       0.76      0.83      0.79       634
        1.0       0.73      0.63      0.68       462

avg / total       0.74      0.75      0.74      1096



In [17]:
knn_best_params = knn_clf.get_best_params()
print(knn_best_params)

{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'}


In [ ]:
#best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
#               'loss': 'mean_squared_error'}

keras_nn_clf = NN_Keras(model_name='keras_nn_wip1', dataset_name='speed_dating')
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


Fitting 5 folds for each of 120 candidates, totalling 600 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\matan\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\matan\Anaconda3\lib\runpy.py

In [ ]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)

None


In [19]:
# keras_nn_clf = NN_Keras(model_name='keras_nn_optimizer_wip0', dataset_name='spam_twitter')
# keras_nn_clf.train(X_train, y_train, optimizer_tuning=True)
# y_pred = keras_nn_clf.predict(X_test)
# keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


In [20]:
# from sklearn.svm import SVC
# 
# linear_svm_clf = SVC(kernel='linear') #optimize kernel, regularization, gamma
# linear_svm_clf.fit(X_train, y_train)
# y_pred = linear_svm_clf.predict(X_test)

In [21]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
svm_clf = SVM(model_name='svm_wip0', dataset_name='speed_dating')
svm_clf.train(X_train, y_train)
y_pred = svm_clf.predict(X_test)
svm_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


ValueError: could not convert string to float: '?'

In [22]:
svm_best_params = svm_clf.get_best_params()
print(svm_best_params) 
#{'svm__degree': 1, 'svm__kernel': 'rbf'}

None
